<a href="https://colab.research.google.com/github/rchavarria3007/ml-ops-pavani/blob/main/gold_ml_attrition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [10]:
# ==============================================================================
# Setup e Carregamento de Dados
# ==============================================================================
import pandas as pd
import numpy as np
from sklearn.preprocessing import PolynomialFeatures, LabelEncoder
from google.colab import files
import io

# Configuração para exibir todas as colunas do pandas
pd.set_option('display.max_columns', None)

# Carregamento do dataset
try:
    file_name = "/content/drive/MyDrive/dev/datascienceexp/sv_hr_final_features_v1.csv"
    sv_hr_features = pd.read_csv(file_name)
    print(f"\nDataset '{file_name}' carregado com sucesso!")
    print("Dimensões do dataset:", sv_hr_features.head())
except StopIteration:
    print("\nNenhum arquivo foi enviado.")
    sv_hr_features = None
except Exception as e:
    print(f"Ocorreu um erro ao carregar o arquivo: {e}")
    sv_hr_features = None




Dataset '/content/drive/MyDrive/dev/datascienceexp/sv_hr_final_features_v1.csv' carregado com sucesso!
Dimensões do dataset:    MaritalStatus_TargetEncoded  IndiceDeBurnout^2  MonthlyIncome^2  \
0                     0.255319               36.0       35916049.0   
1                     0.124814               25.0       26316900.0   
2                     0.255319               25.0        4368100.0   
3                     0.124814               25.0        8462281.0   
4                     0.124814               25.0       12027024.0   

   MonthlyIncome_X_IndiceDeBurnout  YearsAtCompany_X_IndiceDeBurnout  \
0                          35958.0                              36.0   
1                          25650.0                              50.0   
2                          10450.0                               0.0   
3                          14545.0                              40.0   
4                          17340.0                              10.0   

   MonthlyIncome_X_I

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, roc_curve, classification_report
import matplotlib.pyplot as plt
import seaborn as sns # Para visualizações mais agradáveis

# --- 1. Criar um DataFrame de exemplo (Substitua pelo carregamento do seu CSV/Parquet real) ---
# No Colab, você carregaria seu arquivo CSV assim:
# df = pd.read_csv('seu_arquivo_de_dados.csv')
# Exemplo de carregamento do Google Drive:
# from google.colab import drive
# drive.mount('/content/drive')
# file_path = '/content/drive/MyDrive/dados/seu_arquivo.csv'
# df = pd.read_csv(file_path)

data = {
    "id": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15],
    "MaritalStatus": ['Single', 'Married', 'Single', 'Divorced', 'Single', 'Married', 'Single', 'Married', 'Single', 'Married', 'Single', 'Married', 'Single', 'Divorced', 'Single'],
    "JobLevel": [1, 2, 1, 3, 1, 2, 1, 2, 1, 2, 1, 2, 1, 3, 1],
    "JobInvolvement": [1, 2, 1, 3, 1, 2, 1, 2, 1, 2, 1, 2, 1, 3, 1],
    "YearsAtCompany": [3, 5, 2, 8, 4, 6, 1, 7, 3, 9, 5, 10, 2, 6, 4],
    "MonthlyIncome": [3000, 5000, 2500, 7000, 3500, 6000, 2000, 6500, 3200, 8000, 4000, 9000, 2800, 5500, 3800],
    "StockOptionLevel": [0, 1, 0, 2, 0, 1, 0, 1, 0, 1, 0, 1, 0, 2, 0],
    "JobSatisfaction": [3, 4, 2, 3, 2, 4, 1, 3, 2, 4, 2, 4, 1, 3, 2],
    "EnvironmentSatisfaction": [3, 4, 2, 3, 2, 4, 1, 3, 2, 4, 2, 4, 1, 3, 2],
    "Attrition": ['Yes', 'No', 'Yes', 'No', 'Yes', 'No', 'Yes', 'No', 'Yes', 'No', 'Yes', 'No', 'Yes', 'No', 'Yes'],
    "WorkLifeBalance": [1, 2, 1, 3, 1, 2, 1, 2, 1, 2, 1, 2, 1, 3, 1]
}
df = pd.DataFrame(data)

# Mapear 'Yes'/'No' para 1/0 para a variável alvo
df['label'] = df['Attrition'].map({'Yes': 1, 'No': 0})

# Definir as features categóricas e numéricas que identificamos
categorical_features = ['MaritalStatus'] # Adicione outras como EducationField, JobRole, OverTime se for usar
numerical_features = [
    'JobLevel', 'JobInvolvement', 'YearsAtCompany', 'MonthlyIncome',
    'StockOptionLevel', 'JobSatisfaction', 'EnvironmentSatisfaction', 'WorkLifeBalance'
]

# Separar features (X) e target (y)
X = df[categorical_features + numerical_features]
y = df['label']

# --- 2. Dividir os dados em treino e teste ---
# stratify=y é crucial para problemas de classes desbalanceadas como turnover
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)


# --- 3. Criar transformadores para pré-processamento ---
# ColumnTransformer aplica diferentes transformações a diferentes colunas
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features), # Escalonamento para features numéricas
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features) # One-hot encoding para features categóricas
    ])

# --- 4. Construir o Pipeline com pré-processamento e o modelo de Regressão Logística ---
# O Pipeline encadeia as etapas de processamento e o modelo
model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(solver='liblinear', random_state=42)) # 'liblinear' é um bom solver para datasets menores/médios e é robusto
])

# --- 5. Treinar o modelo ---
model_pipeline.fit(X_train, y_train)

# --- 6. Fazer previsões no conjunto de teste ---
# Probabilidades da classe positiva (Attrition = Yes)
y_pred_proba = model_pipeline.predict_proba(X_test)[:, 1]

# Previsões binárias (0 ou 1) usando o threshold padrão (0.5)
y_pred = model_pipeline.predict(X_test)

# --- 7. Avaliar o modelo ---
print("--- Relatório de Classificação ---")
print(classification_report(y_test, y_pred))

print("\n--- AUC (Area Under the ROC Curve) ---")
auc = roc_auc_score(y_test, y_pred_proba)
print(f"AUC: {auc:.4f}")

# Exemplo de como visualizar as probabilidades para alguns funcionários de teste
print("\n--- Probabilidades de Attrition para Dados de Teste (Amostra) ---")
# Criar um DataFrame com as previsões para facilitar a visualização
# Recriamos o índice para que ele combine com o y_test após o split
X_test_display = X_test.copy()
X_test_display['True_Attrition'] = y_test.values # .values para garantir o alinhamento
X_test_display['Predicted_Attrition_Prob'] = y_pred_proba
X_test_display['Predicted_Attrition'] = y_pred

# Ordenar por probabilidade para ver os de "alta chance"
results_df_sorted = X_test_display.sort_values(by='Predicted_Attrition_Prob', ascending=False)
print(results_df_sorted.head(5).to_string()) # Mostrar os 5 com maior probabilidade

# --- Visualizações Adicionais (Recomendadas para o MBA) ---

# Distribuição das Probabilidades
plt.figure(figsize=(10, 6))
sns.histplot(y_pred_proba[y_test == 0], color='blue', label='No Attrition', kde=True, stat='density', alpha=0.5)
sns.histplot(y_pred_proba[y_test == 1], color='red', label='Attrition', kde=True, stat='density', alpha=0.5)
plt.title('Distribuição das Probabilidades de Attrition')
plt.xlabel('Probabilidade de Attrition')
plt.ylabel('Densidade')
plt.legend()
plt.grid(axis='y', alpha=0.75)
plt.show()

# Curva ROC
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='orange', label=f'Curva ROC (AUC = {auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', linestyle='--', label='Classificador Aleatório')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Taxa de Falsos Positivos (FPR)')
plt.ylabel('Taxa de Verdadeiros Positivos (TPR)')
plt.title('Curva Característica de Operação do Receptor (ROC)')
plt.legend(loc="lower right")
plt.grid(True)
plt.show()